In [1]:
# 9/30/2021 - Building A Quantitative Momentum Investing Strategy

# Algorithmic Trading Using Python - Full Course
# https://www.youtube.com/watch?v=xfzGZB4HhEE
# https://iexcloud.io/docs/api/

# https://github.com/nickmccullum/algorithmic-trading-python

import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as score
import xlsxwriter

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

In [3]:
# 1st API call
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
# data.status_code
data

{'companyName': 'Apple Inc',
 'marketcap': 2344068875401,
 'week52high': 159.72,
 'week52low': 107.12,
 'week52highSplitAdjustOnly': 164.59,
 'week52lowSplitAdjustOnly': 110.94,
 'week52change': 0.2215218666764878,
 'sharesOutstanding': 16827368268,
 'float': 0,
 'avg10Volume': 79313420,
 'avg30Volume': 87636468,
 'day200MovingAvg': 138.72,
 'day50MovingAvg': 154.2,
 'employees': 150320,
 'ttmEPS': 5.2,
 'ttmDividendRate': 0.8738376894205913,
 'dividendYield': 0.006209102222475495,
 'nextDividendDate': '',
 'exDividendDate': '2021-08-01',
 'nextEarningsDate': '2021-10-17',
 'peRatio': 27.295221909640883,
 'beta': 0,
 'maxChangePercent': 54.86586614557643,
 'year5ChangePercent': 4.44144680357347,
 'year2ChangePercent': 1.638271200618309,
 'year1ChangePercent': 0.2358865991012537,
 'ytdChangePercent': 0.07362478116724488,
 'month6ChangePercent': 0.16674229966284415,
 'month3ChangePercent': 0.03582067593546307,
 'month1ChangePercent': -0.07114786761534514,
 'day30ChangePercent': -0.068376

In [4]:
data['year1ChangePercent']

0.2358865991012537

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

#for symbol_string in symbol_strings:
#    print(symbol_string)

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['price'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)

final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,161.97,0.595449,N/A
1,AAL,22.48,0.672006,N/A
2,AAP,210.00,0.39459,N/A
3,AAPL,144.93,0.235593,N/A
4,ABBV,114.51,0.301521,N/A
...,...,...,...,...
500,YUM,123.70,0.376093,N/A
501,ZBH,151.57,0.085463,N/A
502,ZBRA,516.27,1.046049,N/A
503,ZION,66.71,1.214332,N/A


In [7]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:50]
# len(final_dataframe[:50])
# final_dataframe
# final_dataframe.reset_index()
final_dataframe.reset_index(inplace = True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,148,DVN,38.11,2.888636,N/A
1,317,MRO,14.47,2.401996,N/A
2,275,LB,81.40,2.362234,N/A
3,175,FANG,103.45,2.309736,N/A
4,299,MCHP,153.89,2.08828,N/A
5,357,OXY,32.07,2.000555,N/A
6,111,COTY,8.47,1.99437,N/A
7,410,SIVB,673.16,1.700368,N/A
8,272,KSS,50.86,1.57948,N/A
9,23,ALB,220.09,1.523678,N/A


In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio:')
    
    try:
        float(portfolio_size)
    except ValueError:
        print('That is not a number! \nPlease try again:')
        portfolio_size = input('Enter the size of your portfolio:')

portfolio_input()
print(portfolio_size)

Enter the size of your portfolio:1000000
1000000


In [9]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe)):
#     final_dataframe.loc[i, 'Number of Shares to Buy'] = 0
# final_dataframe

    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe.loc[i, 'Price'])
final_dataframe

C:\Users\unpat\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,148,DVN,38.11,2.888636,524
1,317,MRO,14.47,2.401996,1382
2,275,LB,81.40,2.362234,245
3,175,FANG,103.45,2.309736,193
4,299,MCHP,153.89,2.08828,129
5,357,OXY,32.07,2.000555,623
6,111,COTY,8.47,1.99437,2361
7,410,SIVB,673.16,1.700368,29
8,272,KSS,50.86,1.57948,393
9,23,ALB,220.09,1.523678,90


In [10]:
# high quality momentum stocks (hqm)

hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

convert_none = lambda x : 0 if x is None else x

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]['price'],
                'N/A',
                convert_none(data[symbol]['stats']['year1ChangePercent']),
                'N/A',
                convert_none(data[symbol]['stats']['month6ChangePercent']),
                'N/A',
                convert_none(data[symbol]['stats']['month3ChangePercent']),
                'N/A',
                convert_none(data[symbol]['stats']['month1ChangePercent']),
                'N/A',
                'N/A'
            ],
            index = hqm_columns),
            ignore_index = True
        )
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,156.19,N/A,0.579302,N/A,0.254069,N/A,0.069160,N/A,-0.107311,N/A,N/A
1,AAL,21.96,N/A,0.690748,N/A,-0.143335,N/A,-0.032572,N/A,0.030404,N/A,N/A
2,AAP,209.00,N/A,0.389144,N/A,0.154456,N/A,0.023406,N/A,0.035045,N/A,N/A
3,AAPL,146.01,N/A,0.238790,N/A,0.162133,N/A,0.034870,N/A,-0.069622,N/A,N/A
4,ABBV,111.99,N/A,0.306369,N/A,0.020373,N/A,-0.032181,N/A,-0.107905,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,128.01,N/A,0.363322,N/A,0.146272,N/A,0.070023,N/A,-0.067611,N/A,N/A
501,ZBH,154.53,N/A,0.084710,N/A,-0.085157,N/A,-0.089240,N/A,-0.026537,N/A,N/A
502,ZBRA,516.08,N/A,1.059777,N/A,0.064885,N/A,-0.027320,N/A,-0.127193,N/A,N/A
503,ZION,65.60,N/A,1.191884,N/A,0.147357,N/A,0.179683,N/A,0.071413,N/A,N/A


In [11]:
# https://stackoverflow.com/questions/65174575/typeerror-not-supported-between-instances-of-nonetype-and-float

time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:

        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        
#        if hqm_dataframe.loc[row, change_col] == None:
#            hqm_dataframe.loc[row, change_col] = 0           
        hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col]) / 100

hqm_dataframe


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,156.19,N/A,0.579302,0.738614,0.254069,0.89703,0.069160,0.827723,-0.107311,0.10495,N/A
1,AAL,21.96,N/A,0.690748,0.8,-0.143335,0.071287,-0.032572,0.372277,0.030404,0.871287,N/A
2,AAP,209.00,N/A,0.389144,0.556436,0.154456,0.754455,0.023406,0.651485,0.035045,0.883168,N/A
3,AAPL,146.01,N/A,0.238790,0.372277,0.162133,0.762376,0.034870,0.69703,-0.069622,0.342574,N/A
4,ABBV,111.99,N/A,0.306369,0.437624,0.020373,0.417822,-0.032181,0.376238,-0.107905,0.10099,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,128.01,N/A,0.363322,0.516832,0.146272,0.732673,0.070023,0.831683,-0.067611,0.364356,N/A
501,ZBH,154.53,N/A,0.084710,0.184158,-0.085157,0.154455,-0.089240,0.188119,-0.026537,0.641584,N/A
502,ZBRA,516.08,N/A,1.059777,0.928713,0.064885,0.544554,-0.027320,0.394059,-0.127193,0.039604,N/A
503,ZION,65.60,N/A,1.191884,0.954455,0.147357,0.736634,0.179683,0.972277,0.071413,0.930693,N/A


In [12]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

hqm_dataframe
    

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,156.19,N/A,0.579302,0.738614,0.254069,0.89703,0.069160,0.827723,-0.107311,0.10495,0.642079
1,AAL,21.96,N/A,0.690748,0.8,-0.143335,0.071287,-0.032572,0.372277,0.030404,0.871287,0.528713
2,AAP,209.00,N/A,0.389144,0.556436,0.154456,0.754455,0.023406,0.651485,0.035045,0.883168,0.711386
3,AAPL,146.01,N/A,0.238790,0.372277,0.162133,0.762376,0.034870,0.69703,-0.069622,0.342574,0.543564
4,ABBV,111.99,N/A,0.306369,0.437624,0.020373,0.417822,-0.032181,0.376238,-0.107905,0.10099,0.333168
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,128.01,N/A,0.363322,0.516832,0.146272,0.732673,0.070023,0.831683,-0.067611,0.364356,0.611386
501,ZBH,154.53,N/A,0.084710,0.184158,-0.085157,0.154455,-0.089240,0.188119,-0.026537,0.641584,0.292079
502,ZBRA,516.08,N/A,1.059777,0.928713,0.064885,0.544554,-0.027320,0.394059,-0.127193,0.039604,0.476733
503,ZION,65.60,N/A,1.191884,0.954455,0.147357,0.736634,0.179683,0.972277,0.071413,0.930693,0.898515


In [13]:
hqm_dataframe.sort_values('HQM Score', ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index( drop = True, inplace = True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MCHP,155.46,N/A,2.022411,0.990099,1.028374,1.0,1.106085,1.0,0.988019,1.0,0.997525
1,DVN,37.08,N/A,2.944239,1.0,0.660584,0.994059,0.228321,0.982178,0.215218,0.990099,0.991584
2,LB,81.09,N/A,2.385963,0.99604,0.829768,0.99802,0.218584,0.980198,0.082580,0.946535,0.980198
3,SIVB,667.85,N/A,1.712245,0.986139,0.322811,0.962376,0.170236,0.968317,0.163602,0.982178,0.974752
4,PWR,120.12,N/A,1.201083,0.956436,0.296111,0.938614,0.259305,0.988119,0.117692,0.970297,0.963366
5,COP,71.00,N/A,1.149505,0.944554,0.303993,0.946535,0.123469,0.926733,0.221347,0.992079,0.952475
6,IT,320.80,N/A,1.475656,0.978218,0.665068,0.99604,0.267368,0.990099,-0.015932,0.70495,0.917327
7,CF,62.31,N/A,0.880787,0.883168,0.254129,0.89901,0.093906,0.881188,0.229001,0.994059,0.914356
8,CMA,84.91,N/A,1.245909,0.962376,0.148287,0.738614,0.138955,0.946535,0.102099,0.960396,0.90198
9,ZION,65.60,N/A,1.191884,0.954455,0.147357,0.736634,0.179683,0.972277,0.071413,0.930693,0.898515


In [14]:
portfolio_input()

Enter the size of your portfolio:1000000


In [15]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)

for i in hqm_dataframe.index:
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe.loc[i, 'Price'])
    
hqm_dataframe


C:\Users\unpat\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MCHP,155.46,128,2.022411,0.990099,1.028374,1.0,1.106085,1.0,0.988019,1.0,0.997525
1,DVN,37.08,539,2.944239,1.0,0.660584,0.994059,0.228321,0.982178,0.215218,0.990099,0.991584
2,LB,81.09,246,2.385963,0.99604,0.829768,0.99802,0.218584,0.980198,0.082580,0.946535,0.980198
3,SIVB,667.85,29,1.712245,0.986139,0.322811,0.962376,0.170236,0.968317,0.163602,0.982178,0.974752
4,PWR,120.12,166,1.201083,0.956436,0.296111,0.938614,0.259305,0.988119,0.117692,0.970297,0.963366
5,COP,71.00,281,1.149505,0.944554,0.303993,0.946535,0.123469,0.926733,0.221347,0.992079,0.952475
6,IT,320.80,62,1.475656,0.978218,0.665068,0.99604,0.267368,0.990099,-0.015932,0.70495,0.917327
7,CF,62.31,320,0.880787,0.883168,0.254129,0.89901,0.093906,0.881188,0.229001,0.994059,0.914356
8,CMA,84.91,235,1.245909,0.962376,0.148287,0.738614,0.138955,0.946535,0.102099,0.960396,0.90198
9,ZION,65.60,304,1.191884,0.954455,0.147357,0.736634,0.179683,0.972277,0.071413,0.930693,0.898515


In [16]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine = 'xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name = "Momentum Strategy", index = False)


In [17]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [18]:
column_formats = {
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['One-Year Price Return', percent_template],
    'E': ['One-Year Return Percentile', percent_template],
    'F': ['Six-Month Price Return', percent_template],
    'G': ['Six-Month Return Percentile', percent_template],
    'H': ['Three-Month Price Return', percent_template],
    'I': ['Three-Month Return Percentile', percent_template],
    'J': ['One-Month Price Return', percent_template],
    'K': ['One-Month Return Percentile', percent_template],
    'L': ['HQM Score', percent_template]
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])


In [19]:
writer.save()